## 8 AIS dataset process - for the RDA (Relative displacement angle) approach 

In [333]:
# from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
from haversine import haversine, Unit


pd.options.mode.chained_assignment = None  # default='warn'

In [334]:
load_X = np.load(r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_5_output\data_2019_12_v5_out\data_2019_12_X.npy')
load_Y = np.load(r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_5_output\data_2019_12_v5_out\data_2019_12_Y.npy')
to_store_path =  r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_10_output_encounter_unifom_ts\data_2019_02_v10_out_encounters_uniform_ts.csv'

Y_distance_path = r"W:/Neeraj_thesis/version2_thesis/dataset/2019/generator_version_5_output/data_2019_12_v5_out/data_2019_12_Y_array_distance_m_s.npy"
Y_angle_path= r"W:/Neeraj_thesis/version2_thesis/dataset/2019/generator_version_5_output/data_2019_12_v5_out/data_2019_12_Y_array_angle_m.npy"


In [336]:
X_flat = load_X.flatten()
Y_flat = load_Y.flatten()
X_array = X_flat.reshape(-1,8)
Y_array = Y_flat.reshape(-1,2)

X_df = pd.DataFrame(X_array, columns= ['lat_s', 'lon_s', 'sog_s', 'cog_sin' , 'cog_cos', 'NDC_s', 'In_TSS', 'Vessel_TSS'])
Y_df = pd.DataFrame(Y_array, columns= ['lat_s', 'lon_s'])

In [337]:
min_lat, max_lat = 58.6, 59.93
min_lon, max_lon = 9.4, 11.45

X_df['lat'] = X_df['lat_s'] * (max_lat - min_lat) + min_lat
X_df['lon'] = X_df['lon_s'] * (max_lon - min_lon) + min_lon

Y_df['lat'] = Y_df['lat_s'] * (max_lat - min_lat) + min_lat
Y_df['lon'] = Y_df['lon_s'] * (max_lon - min_lon) + min_lon

In [338]:
Y_df['lat_ref'] = 0
Y_df['lon_ref'] = 0

In [339]:
start_indices = np.arange(0, len(Y_df), 10)
end_indices = start_indices + 9

lat_val = X_df['lat'].values[end_indices]
lon_val = X_df['lon'].values[end_indices]

lat_rep = np.repeat(lat_val, 10)
lon_rep = np.repeat(lon_val, 10)

Y_df['lat_ref'] = lat_rep[:len(Y_df)]
Y_df['lon_ref'] = lon_rep[:len(Y_df)]

In [341]:
from haversine import haversine, Unit 
import math

# calculate distance using apply function
Y_df['distance_m'] = Y_df.apply(lambda row: haversine((row['lat_ref'], row['lon_ref']), (row['lat'], row['lon']), unit=Unit.METERS), axis=1)
# calculate angle (bearing) using apply function
Y_df['angle'] = Y_df.apply(lambda row: math.degrees(math.atan2(math.sin(math.radians(row['lon'] - row['lon_ref'])) * math.cos(math.radians(row['lat'])), math.cos(math.radians(row['lat_ref'])) * math.sin(math.radians(row['lat'])) - math.sin(math.radians(row['lat_ref'])) * math.cos(math.radians(row['lat'])) * math.cos(math.radians(row['lon'] - row['lon_ref'])))), axis=1)


In [343]:
# print('max distance:', Y_df['distance_m'].max()) 
# print('min distance:', Y_df['distance_m'].min())

max distance: 5645.264070498614
min distance: 0.0


In [345]:
## to predict distance and angle 
Y_df_v2 = Y_df.copy()

# define the min and max values for lat and lon standardization
min_distance_m, max_distance_m = 0, 6000

# standardize the lat and lon values between 0 and 1
Y_df_v2['distance_m_s'] = (Y_df_v2['distance_m'] - min_distance_m) / (max_distance_m - min_distance_m)

Y_df_v2 = Y_df_v2.drop(['lat_s','lon_s','lat','lon','lat_ref','lon_ref','distance_m'], axis=1).reset_index(drop = True)

# # New column arrangement
# col_arrange = ['lat_s','lon_s','sog_s','cog_sin','cog_cos','NDC_s','In_TSS', 'Vessel_TSS', 'new_short_voyage_id','position']
# ais_std_v3 = ais_std_v3[col_arrange]

In [347]:
# Y_df_angle_dist_from last input value
# Generate sine and cosine columns

Y_df_v2['sin_angle_m'] = Y_df_v2['angle'].apply(lambda x: math.sin(math.radians(x)))
Y_df_v2['cos_angle_m'] = Y_df_v2['angle'].apply(lambda x: math.cos(math.radians(x)))


In [349]:
Y_df_v3 = Y_df_v2.drop(['angle'], axis=1).reset_index(drop = True)
Y_df_distance_m_s = Y_df_v3['distance_m_s']
Y_df_angle_m = Y_df_v3[['sin_angle_m', 'cos_angle_m']]

In [351]:
# Reshape the array
Y_array_distance_m_s = Y_df_distance_m_s.values.reshape(len(Y_df_distance_m_s)//10, 10)
# Reshape the angle dataset
Y_array_angle_m = Y_df_angle_m.values.reshape(len(Y_df_angle_m)//10, 10, 2)

In [353]:
# Save the arrays
np.save(Y_distance_path, Y_array_distance_m_s)
np.save(Y_angle_path, Y_array_angle_m)